In [0]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from numpy import linalg
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
def mean(Xtrain):
    return np.mean(Xtrain,axis = 1)
#     mean = [0] * Xtrain.shape[0]
#     mean = np.array(mean)
#     for col in Xtrain.T:
#         mean = mean + col
#     N = Xtrain.shape[1]
#     mean = mean/N
#     return mean

In [0]:
# a = [0] * 10
# a = np.array(a)
# X = np.arange(8)
# print((a.T).shape)

In [0]:
def X(Xtrain, mean):
    Xdef = []
    for col in Xtrain.T:
        Xdef.append(col - mean)
    Xdef = np.array(Xdef)
    return Xdef.T

In [0]:
def covMat(X):
#     C1 = np.dot(X, X.T)
    C2 = np.cov(X)
    return C2

In [0]:
def eigenVectors(X, C):
    eigVal1, eigVect1 = linalg.eig(C)
    return eigVect1
# def eigenVectors(X, C):
#     eigVal2, eigVect2 = linalg.eig(C)
#     print("EigVect2: ", eigVect2.shape)
#     eigVect1 = np.matmul(X, eigVect2)
#     print("EigVect1: ", eigVect1.shape)
#     for i in range(0, ((eigVect1.T).shape)[0]):
#         (eigVect1.T)[i] = ((eigVect1.T)[i])/eigVal2[i]
#     return eigVect1

In [0]:
def eigenWeights(E, x):
    weights = (E.T).dot(x)
    return weights

In [0]:
def face_detection(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cascade_image = cv2.CascadeClassifier('/haarcascade_frontalface_alt.xml')
    faces = cascade_image.detectMultiScale(image, scaleFactor = 1.3, minNeighbors = 5)
    if (len(faces) == 0):
        return None, None
    (x, y, l, b) = faces[0]
    return image[y:y+b, x:x+l], faces[0]

In [0]:
def loadDataLabels(data_path):
    faces = []
    labels = []
    enc_labels = []
    directories = os.listdir(data_path)
    for i, direc in enumerate(directories):
        sub_dir_path = data_path + "/" + direc
        sub_dir_images = os.listdir(sub_dir_path)
        label = direc
        for j, image_name in enumerate(sub_dir_images):
            image_path = sub_dir_path + "/" + image_name
            img = cv2.imread(image_path)
#             image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            face_detected, shape = face_detection(img)
            if face_detected is not None:
                face_detected = cv2.resize(face_detected, (256, 256))
                faces.append(face_detected)
                labels.append(label)
#                 enc_labels.appen(c)
#             if image is not None:
#                 image = cv2.resize(image, (256, 256))
#                 faces.append(image)
#                 labels.append(label)
    labels = np.array(labels)
    label_encoder = preprocessing.LabelEncoder()
    enc_labels = label_encoder.fit_transform(labels) 
    labels_unique = label_encoder.classes_
    faces = np.array(faces)
    labels_unique = np.array(labels_unique)
    enc_labels = np.array(enc_labels)
    return faces, labels, labels_unique, enc_labels

In [0]:
def weightMatrix(X_data):
#     print('X_data: ', X_data)
    print ('X_data shape: ', X_data.shape)
    mean_ = mean(X_data)
#     print('mean: ', mean_)
    print ('mean shape: ', mean_.shape)
    X_ = X(X_data, mean_)
#     print('X: ', X_)
    print ('X shape: ', X_.shape)
    C = covMat(X_)
#     print('C: ', C)
    print ('C shape: ', C.shape)
#     E = eigenVectors(X_)
    E = eigenVectors(X_, C)
#     print('E: ', E)
    print ('E shape: ', E.shape)
#     weightMatrix = eigenWeights(E, (X_data.T)[0])
#     for i in range(1, (X_data.T).shape[0]):
# #     for x_col in (X_data.T):
#         weightMatrix = np.vstack((weightMatrix, eigenWeights(E, ((X_data.T)[i]))))
    weightMatrix = np.matmul(E.T, X_data)
#     weightMatrix = eigenWeights(E, (X_data.T)[index])
#     print('wtMat: ', weightMatrix)
    print ('wtMat shape: ', weightMatrix.shape)
    return E, weightMatrix

In [0]:
def flattenData(X):
    X_data = []
    for image in X:
        X_data.append(image.flatten())
    X_data = np.array(X_data)
    return X_data.T

In [0]:
def PCAfamilyCoeff(X_train):
    X_data = flattenData(X_train)
    eigenVect, eigenCoeff = weightMatrix(X_data)
    return eigenVect, eigenCoeff

In [0]:
def drawEigenFaces(eigenVect, index):
    eigenFace = ((eigenVect.T)[index]).reshape(256, 256)
    plt.imshow(eigenFace)
    plt.show()

In [0]:
def euclideanDistance(vector1, vector2):
    return np.linalg.norm(vector1 - vector2)

In [0]:
def leastEucDist(matrix, vector):
    max_ind = -1
    max_ed = -1
    for i, col in enumerate(((matrix.T).shape)[0]):
        euc_dist = euclideanDistance(vector, col) 
        if(max_ed < euc_dist):
            max_ed = euc_dist
            max_ind = i
    return max_ind

In [0]:
def PCA_prediction(X_train, y_train, X_test):
    train_eigenVect, train_eigenCoeff = PCAfamilyCoeff(X_train)
    test_eigenVect, test_eigenCoeff = PCAfamilyCoeff(X_test)
    predictions = []
    for test_col in test_eigenCoeff.T:
        predictions.append(leastEucDist(train_eigenCoeff, test_col))
    return predictions

In [0]:
def PCA_accuracy(predictions, y_test):
    count = 0
    if (len(predictions) != len(y_test)):
        print("Error...Mismatch in Lenghts!")
    for i in range(len(predictions)):
        if (predictions[i] == y_test[i]):
            count += 1
    print("PCA implementation Accuracy: ", (count/len(predictions))*100)

In [0]:
def Example(index, predictions, y_test, unique_labels):
    plt.imshow(X_test[index])
    plt.show()
    print("Prediction made by Model: ", unique_labels[predictions[index]])
    print("Ground Truth: ", unique_labels[y_test])

In [0]:
faces, labels, unique_labels, enc_labels = loadDataLabels("/content/drive/My Drive/SampleVRdata")
(X_train, X_test, y_train, y_test) = train_test_split(faces, enc_labels, test_size = 0.25, random_state = 42)

In [0]:
eigenVect, eigenCoeff = PCAfamilyCoeff(X_train)
drawEigenFaces(eigenVect, 10)

In [0]:
predictions = PCA_prediction(X_train, y_train, X_test)
PCA_accuracy(predictions, y_test)
Example(100, predictions, y_test, unique_labels)

In [0]:
mean_ = mean(X_train)
# X(X_train, mean_)
print(mean_)

In [0]:
a = np.random.randn(9, 6) + 1j*np.random.randn(9, 6)
print(a.shape)
u, s, vh = np.linalg.svd(a, full_matrices=True)
print(X_train[30])    

In [0]:
PCAfamilyCoeff([X_train[0], X_train[30]])

In [0]:
exd = flattenData(X_train)
mean1 = mean(exd)
X1 = X(exd, mean1)
print(X1.shape)
print(X1)

In [0]:
C1 = covMat(X1)
# print(C1)
E1 = eigenVectors(X1, C1)
# print(E1)

In [0]:
drawEigenFaces(E1, 10)